# PGURE-SVT HyperSpy Demonstration

### Tom Furnival ([tjof2@cam.ac.uk](mailto:tjof2@cam.ac.uk))

HyperSpy is an open-source Python library that makes signal handling and processing straightforward in Python, with a friendly API. 

While you can use `pguresvt.pguresvt.SVT` to denoise a numpy array directly, `pguresvt.hspysvt.HSPYSVT` can directly denoise a HyperSpy signal.

In [ ]:
# Configures the plotting backend
#%matplotlib inline
%matplotlib qt

In [ ]:
import numpy as np
# Import the HyperSpy API
import hyperspy.api as hs

In [ ]:
# Import the HyperSpy wrapper for PGURE-SVT
from pguresvt import hspysvt

## 1. Simulated dataset

First, we load the simulated dataset using HyperSpy.

In [ ]:
# Load example dataset
movie = hs.load("../test/examplesequence.tif")

# Truncate to 25 frames, and plot the result
movie = movie.inav[:25]
movie.plot(navigator='slider')

Now we corrupt it with a mixture of Poisson and Gaussian noise.

In [ ]:
# First extract the data and rescale to [0,1] range
clean = movie._data_aligned_with_axes
clean = clean / np.amax(clean)

# Detector gain
gain = 0.1
# Detector offset
offset = 0.1
# Detector variance
sigma = 0.1

def addnoise(x):
    return gain * np.random.poisson(x / gain) + offset + sigma * np.random.randn()
addnoise = np.vectorize(addnoise, otypes=[np.float])

noisy = addnoise(clean)

noisy_movie = hs.signals.Image(noisy)
noisy_movie.plot()

Next we initialise the SVT denoising function. The full list of options (with default values) is:

```
hspysvt.HSPYSVT(patchsize=4,
                patchoverlap=1,
                length=15,
                optimize=True,
                threshold=0.5,
                estimatenoise=True,
                alpha=-1., 
                mu=-1., 
                sigma=-1.,                
                arpssize=7, 
                tol=1e-7,
                median=5,
                hotpixelthreshold=10)
```

In [ ]:
# Initialize with default parameters
svt = hspysvt.HSPYSVT(patchsize=4,
                      patchoverlap=2,
                      length=15,
                      threshold=0.5,
                      tol=1e-6)

Now we are able to run the denoising and plot the result:

In [ ]:
# Run the denoising
denoised_movie = svt.denoise(noisy_movie)

# Plot denoised data
denoised_movie.plot()

### 2. Time-resolved STEM image sequence

First, we load the simulated dataset using HyperSpy, and then corrupt it with a mixture of Poisson and Gaussian noise.
